In [30]:
import pandas as pd
import numpy as np
from collections import Counter
import colorsys
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression


In [2]:
cic_df = pd.read_csv("./data/cic_vectorized.csv")

In [4]:
cic_df.head(5)

,Unnamed: 0,gameid,roundid,round_num,lux_difficulty_label,lux_difficulty_rating,condition,lux_label,matcher_succeeded,split,target,alt1,alt2,clicked,full_text,utterance_events,num_utterances,utterance_events_pattern
0,0,1124-1,1124-1+1,1,single-utterance-partial-match,2,close,darker blue,False,train,"(0.6277777777777778, 0.81, 0.5)","(0.6888888888888889, 0.92, 0.5)","(0.7861111111111111, 0.87, 0.5)",alt1,"[{""text"": ""The darker blue one"", ""subject"": {""...","[{'text': 'The darker blue one', 'subject': {'...",1,S
1,1,1124-1,1124-1+2,2,single-utterance-full-match,0,far,purple,True,train,"(0.6916666666666667, 0.25, 0.5)","(0.15, 0.53, 0.5)","(0.35833333333333334, 0.66, 0.5)",target,"[{""text"": ""purple"", ""subject"": {""worker_id"": 2...","[{'text': 'purple', 'subject': {'worker_id': 2...",1,S
2,2,1124-1,1124-1+3,3,mulitple-utterances-full-match,1,close,medium pink,False,train,"(0.8361111111111111, 0.57, 0.5)","(0.8083333333333333, 0.59, 0.5)","(0.8388888888888889, 0.86, 0.5)",alt1,"[{""text"": ""Medium pink"", ""subject"": {""worker_i...","[{'text': 'Medium pink', 'subject': {'worker_i...",2,SS
3,3,1124-1,1124-1+4,4,single-utterance-full-match,0,split,lime,False,train,"(0.24166666666666667, 0.66, 0.5)","(0.2361111111111111, 0.21, 0.5)","(0.8305555555555556, 0.29, 0.5)",alt1,"[{""text"": ""lime"", ""subject"": {""worker_id"": 201...","[{'text': 'lime', 'subject': {'worker_id': 201...",1,S
4,4,1124-1,1124-1+5,5,single-utterance-full-match,0,far,mint green,True,train,"(0.43333333333333335, 0.84, 0.5)","(0.6638888888888889, 0.87, 0.5)","(0.15833333333333333, 0.64, 0.5)",target,"[{""text"": ""Mint green."", ""subject"": {""worker_i...","[{'text': 'Mint green.', 'subject': {'worker_i...",1,S


In [5]:
### Basic Color Classifier
# restrict dataset to simple colors
# train a classifier on these

In [6]:
simple_df = cic_df[["target","alt1","alt2","lux_label"]]

In [9]:
simple_df

,target,alt1,alt2,lux_label
0,"(0.6277777777777778, 0.81, 0.5)","(0.6888888888888889, 0.92, 0.5)","(0.7861111111111111, 0.87, 0.5)",darker blue
1,"(0.6916666666666667, 0.25, 0.5)","(0.15, 0.53, 0.5)","(0.35833333333333334, 0.66, 0.5)",purple
2,"(0.8361111111111111, 0.57, 0.5)","(0.8083333333333333, 0.59, 0.5)","(0.8388888888888889, 0.86, 0.5)",medium pink
3,"(0.24166666666666667, 0.66, 0.5)","(0.2361111111111111, 0.21, 0.5)","(0.8305555555555556, 0.29, 0.5)",lime
4,"(0.43333333333333335, 0.84, 0.5)","(0.6638888888888889, 0.87, 0.5)","(0.15833333333333333, 0.64, 0.5)",mint green
...,...,...,...,...
47036,"(0.5305555555555556, 0.02, 0.5)","(0.5472222222222223, 0.13, 0.5)","(0.3888888888888889, 0.12, 0.5)",grey
47037,"(0.19166666666666668, 0.94, 0.5)","(0.2361111111111111, 0.78, 0.5)","(0.49722222222222223, 0.41, 0.5)",yellow
47038,"(0.14444444444444443, 0.44, 0.5)","(0.9972222222222222, 0.3, 0.5)","(0.09722222222222222, 0.24, 0.5)",yellow
47039,"(0.7888888888888889, 0.24, 0.5)","(0.48333333333333334, 0.35, 0.5)","(0.8861111111111111, 0.01, 0.5)",purple


In [16]:
simple_nona = simple_df[~simple_df.lux_label.isna()]

In [17]:
simple_nona

,target,alt1,alt2,lux_label
0,"(0.6277777777777778, 0.81, 0.5)","(0.6888888888888889, 0.92, 0.5)","(0.7861111111111111, 0.87, 0.5)",darker blue
1,"(0.6916666666666667, 0.25, 0.5)","(0.15, 0.53, 0.5)","(0.35833333333333334, 0.66, 0.5)",purple
2,"(0.8361111111111111, 0.57, 0.5)","(0.8083333333333333, 0.59, 0.5)","(0.8388888888888889, 0.86, 0.5)",medium pink
3,"(0.24166666666666667, 0.66, 0.5)","(0.2361111111111111, 0.21, 0.5)","(0.8305555555555556, 0.29, 0.5)",lime
4,"(0.43333333333333335, 0.84, 0.5)","(0.6638888888888889, 0.87, 0.5)","(0.15833333333333333, 0.64, 0.5)",mint green
...,...,...,...,...
47036,"(0.5305555555555556, 0.02, 0.5)","(0.5472222222222223, 0.13, 0.5)","(0.3888888888888889, 0.12, 0.5)",grey
47037,"(0.19166666666666668, 0.94, 0.5)","(0.2361111111111111, 0.78, 0.5)","(0.49722222222222223, 0.41, 0.5)",yellow
47038,"(0.14444444444444443, 0.44, 0.5)","(0.9972222222222222, 0.3, 0.5)","(0.09722222222222222, 0.24, 0.5)",yellow
47039,"(0.7888888888888889, 0.24, 0.5)","(0.48333333333333334, 0.35, 0.5)","(0.8861111111111111, 0.01, 0.5)",purple


In [18]:
simple_nona["lux_words"] = simple_nona["lux_label"].apply(lambda x: x.split())

<ipython-input-18-369a094f8697>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple_nona["lux_words"] = simple_nona["lux_label"].apply(lambda x: x.split())


In [19]:
simple_nona

,target,alt1,alt2,lux_label,lux_words
0,"(0.6277777777777778, 0.81, 0.5)","(0.6888888888888889, 0.92, 0.5)","(0.7861111111111111, 0.87, 0.5)",darker blue,"[darker, blue]"
1,"(0.6916666666666667, 0.25, 0.5)","(0.15, 0.53, 0.5)","(0.35833333333333334, 0.66, 0.5)",purple,[purple]
2,"(0.8361111111111111, 0.57, 0.5)","(0.8083333333333333, 0.59, 0.5)","(0.8388888888888889, 0.86, 0.5)",medium pink,"[medium, pink]"
3,"(0.24166666666666667, 0.66, 0.5)","(0.2361111111111111, 0.21, 0.5)","(0.8305555555555556, 0.29, 0.5)",lime,[lime]
4,"(0.43333333333333335, 0.84, 0.5)","(0.6638888888888889, 0.87, 0.5)","(0.15833333333333333, 0.64, 0.5)",mint green,"[mint, green]"
...,...,...,...,...,...
47036,"(0.5305555555555556, 0.02, 0.5)","(0.5472222222222223, 0.13, 0.5)","(0.3888888888888889, 0.12, 0.5)",grey,[grey]
47037,"(0.19166666666666668, 0.94, 0.5)","(0.2361111111111111, 0.78, 0.5)","(0.49722222222222223, 0.41, 0.5)",yellow,[yellow]
47038,"(0.14444444444444443, 0.44, 0.5)","(0.9972222222222222, 0.3, 0.5)","(0.09722222222222222, 0.24, 0.5)",yellow,[yellow]
47039,"(0.7888888888888889, 0.24, 0.5)","(0.48333333333333334, 0.35, 0.5)","(0.8861111111111111, 0.01, 0.5)",purple,[purple]


In [20]:
simple_nona["lux_last"] = simple_nona["lux_label"].apply(lambda x: x.split()[-1])

<ipython-input-20-1ca215c649dd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simple_nona["lux_last"] = simple_nona["lux_label"].apply(lambda x: x.split()[-1])


In [21]:
simple_nona

,target,alt1,alt2,lux_label,lux_words,lux_last
0,"(0.6277777777777778, 0.81, 0.5)","(0.6888888888888889, 0.92, 0.5)","(0.7861111111111111, 0.87, 0.5)",darker blue,"[darker, blue]",blue
1,"(0.6916666666666667, 0.25, 0.5)","(0.15, 0.53, 0.5)","(0.35833333333333334, 0.66, 0.5)",purple,[purple],purple
2,"(0.8361111111111111, 0.57, 0.5)","(0.8083333333333333, 0.59, 0.5)","(0.8388888888888889, 0.86, 0.5)",medium pink,"[medium, pink]",pink
3,"(0.24166666666666667, 0.66, 0.5)","(0.2361111111111111, 0.21, 0.5)","(0.8305555555555556, 0.29, 0.5)",lime,[lime],lime
4,"(0.43333333333333335, 0.84, 0.5)","(0.6638888888888889, 0.87, 0.5)","(0.15833333333333333, 0.64, 0.5)",mint green,"[mint, green]",green
...,...,...,...,...,...,...
47036,"(0.5305555555555556, 0.02, 0.5)","(0.5472222222222223, 0.13, 0.5)","(0.3888888888888889, 0.12, 0.5)",grey,[grey],grey
47037,"(0.19166666666666668, 0.94, 0.5)","(0.2361111111111111, 0.78, 0.5)","(0.49722222222222223, 0.41, 0.5)",yellow,[yellow],yellow
47038,"(0.14444444444444443, 0.44, 0.5)","(0.9972222222222222, 0.3, 0.5)","(0.09722222222222222, 0.24, 0.5)",yellow,[yellow],yellow
47039,"(0.7888888888888889, 0.24, 0.5)","(0.48333333333333334, 0.35, 0.5)","(0.8861111111111111, 0.01, 0.5)",purple,[purple],purple


In [23]:
color_words = Counter(simple_nona.lux_last)

In [24]:
len(color_words)

159

In [25]:
color_words.most_common(50)

[('green', 8821),
 ('purple', 6663),
 ('blue', 6630),
 ('grey', 4193),
 ('pink', 3328),
 ('yellow', 1736),
 ('orange', 1374),
 ('red', 1373),
 ('brown', 1283),
 ('teal', 460),
 ('tan', 319),
 ('blueish', 245),
 ('grass', 232),
 ('greyish', 229),
 ('sky', 221),
 ('greenish', 215),
 ('brownish', 211),
 ('aqua', 202),
 ('olive', 192),
 ('pinkish', 159),
 ('mauve', 159),
 ('yellowish', 147),
 ('magenta', 125),
 ('lime', 120),
 ('purplish', 120),
 ('rose', 110),
 ('turquoise', 105),
 ('reddish', 104),
 ('lavender', 104),
 ('mustard', 102),
 ('violet', 92),
 ('gold', 74),
 ('beige', 73),
 ('salmon', 71),
 ('barney', 68),
 ('grape', 51),
 ('orangish', 41),
 ('fuchsia', 41),
 ('ocean', 38),
 ('drab', 36),
 ('white', 36),
 ('black', 35),
 ('pumpkin', 33),
 ('sand', 33),
 ('peach', 33),
 ('tealish', 28),
 ('seafoam', 28),
 ('royal', 25),
 ('maroon', 24),
 ('mint', 24)]

In [26]:
## Train a binary classifier for each color word

In [50]:
traindf = simple_nona[:40000]
testdf = simple_nona[40000:]


In [28]:
traindf

,target,alt1,alt2,lux_label,lux_words,lux_last
0,"(0.6277777777777778, 0.81, 0.5)","(0.6888888888888889, 0.92, 0.5)","(0.7861111111111111, 0.87, 0.5)",darker blue,"[darker, blue]",blue
1,"(0.6916666666666667, 0.25, 0.5)","(0.15, 0.53, 0.5)","(0.35833333333333334, 0.66, 0.5)",purple,[purple],purple
2,"(0.8361111111111111, 0.57, 0.5)","(0.8083333333333333, 0.59, 0.5)","(0.8388888888888889, 0.86, 0.5)",medium pink,"[medium, pink]",pink
3,"(0.24166666666666667, 0.66, 0.5)","(0.2361111111111111, 0.21, 0.5)","(0.8305555555555556, 0.29, 0.5)",lime,[lime],lime
4,"(0.43333333333333335, 0.84, 0.5)","(0.6638888888888889, 0.87, 0.5)","(0.15833333333333333, 0.64, 0.5)",mint green,"[mint, green]",green
...,...,...,...,...,...,...
46072,"(0.4222222222222222, 0.52, 0.5)","(0.46111111111111114, 0.48, 0.5)","(0.5388888888888889, 0.83, 0.5)",dark green,"[dark, green]",green
46073,"(0.5861111111111111, 0.92, 0.5)","(0.6, 0.61, 0.5)","(0.41388888888888886, 0.76, 0.5)",bright blue,"[bright, blue]",blue
46074,"(0.5861111111111111, 0.19, 0.5)","(0.5611111111111111, 0.1, 0.5)","(0.5611111111111111, 0.63, 0.5)",purplish grey,"[purplish, grey]",grey
46075,"(0.7305555555555555, 0.33, 0.5)","(0.05277777777777778, 0.88, 0.5)","(0.2916666666666667, 0.88, 0.5)",purple,[purple],purple


In [29]:
#def get_color_classifier(colword):
    
#traindf[traindf.lux_last == "green"]

#X 

#clf = LogisticRegression(random_state=0).fit(X, y)

,target,alt1,alt2,lux_label,lux_words,lux_last
4,"(0.43333333333333335, 0.84, 0.5)","(0.6638888888888889, 0.87, 0.5)","(0.15833333333333333, 0.64, 0.5)",mint green,"[mint, green]",green
7,"(0.3, 0.16, 0.5)","(0.21666666666666667, 0.99, 0.5)","(0.425, 0.12, 0.5)",camo green,"[camo, green]",green
11,"(0.32222222222222224, 0.59, 0.5)","(0.4444444444444444, 0.77, 0.5)","(0.3888888888888889, 0.86, 0.5)",forest green,"[forest, green]",green
18,"(0.3138888888888889, 0.21, 0.5)","(0.4027777777777778, 0.34, 0.5)","(0.9833333333333333, 0.41, 0.5)",camo green,"[camo, green]",green
23,"(0.3333333333333333, 0.39, 0.5)","(0.49722222222222223, 0.65, 0.5)","(0.3638888888888889, 0.81, 0.5)",forest green,"[forest, green]",green
...,...,...,...,...,...,...
46057,"(0.2222222222222222, 0.4, 0.5)","(0.2, 0.84, 0.5)","(0.2611111111111111, 0.45, 0.5)",olive green,"[olive, green]",green
46061,"(0.2777777777777778, 0.26, 0.5)","(0.9194444444444444, 0.8, 0.5)","(0.40555555555555556, 0.97, 0.5)",dark green,"[dark, green]",green
46066,"(0.30833333333333335, 0.58, 0.5)","(0.275, 0.83, 0.5)","(0.39166666666666666, 0.43, 0.5)",bright green,"[bright, green]",green
46069,"(0.3472222222222222, 0.33, 0.5)","(0.8694444444444445, 0.01, 0.5)","(0.4527777777777778, 0.82, 0.5)",green,[green],green


In [52]:
traindf["target_hsv"] = traindf["target"].apply(lambda x: np.array(eval(x)))
testdf["target_hsv"] = testdf["target"].apply(lambda x: np.array(eval(x)))

<ipython-input-52-bf870f994065>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindf["target_hsv"] = traindf["target"].apply(lambda x: np.array(eval(x)))
<ipython-input-52-bf870f994065>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["target_hsv"] = testdf["target"].apply(lambda x: np.array(eval(x)))


In [53]:
X = np.array(list(traindf.target_hsv))
Xtest = np.array(list(testdf.target_hsv))

In [54]:
X.shape

(40000, 3)

In [55]:
y = np.array(traindf.lux_last == "green")
ytest = np.array(testdf.lux_last == "green")

In [56]:
y

array([False, False, False, ..., False, False, False])

In [57]:
clf = LogisticRegression(random_state=0).fit(X, y)

In [58]:
clf.score(Xtest,ytest)

0.7378984651711924

In [59]:
clf.predict_proba(Xtest)

array([[0.95042298, 0.04957702],
       [0.89139192, 0.10860808],
       [0.72260335, 0.27739665],
       ...,
       [0.5857113 , 0.4142887 ],
       [0.94050991, 0.05949009],
       [0.5521994 , 0.4478006 ]])

In [60]:
testdf

,target,alt1,alt2,lux_label,lux_words,lux_last,target_hsv
46077,"(0.7972222222222223, 0.1, 0.5)","(0.8361111111111111, 0.4, 0.5)","(0.8361111111111111, 0.17, 0.5)",grey,[grey],grey,"[0.7972222222222223, 0.1, 0.5]"
46078,"(0.6888888888888889, 0.51, 0.5)","(0.7027777777777777, 0.14, 0.5)","(0.7277777777777777, 0.64, 0.5)",blue,[blue],blue,"[0.6888888888888889, 0.51, 0.5]"
46079,"(0.18611111111111112, 0.04, 0.5)","(0.9666666666666667, 0.04, 0.5)","(0.6472222222222223, 0.65, 0.5)",greenish,[greenish],greenish,"[0.18611111111111112, 0.04, 0.5]"
46080,"(0.011111111111111112, 0.89, 0.5)","(0.9555555555555556, 0.89, 0.5)","(0.5055555555555555, 0.52, 0.5)",orange red,"[orange, red]",red,"[0.011111111111111112, 0.89, 0.5]"
46081,"(0.19722222222222222, 0.05, 0.5)","(0.4444444444444444, 0.2, 0.5)","(0.3055555555555556, 0.27, 0.5)",brown,[brown],brown,"[0.19722222222222222, 0.05, 0.5]"
...,...,...,...,...,...,...,...
47036,"(0.5305555555555556, 0.02, 0.5)","(0.5472222222222223, 0.13, 0.5)","(0.3888888888888889, 0.12, 0.5)",grey,[grey],grey,"[0.5305555555555556, 0.02, 0.5]"
47037,"(0.19166666666666668, 0.94, 0.5)","(0.2361111111111111, 0.78, 0.5)","(0.49722222222222223, 0.41, 0.5)",yellow,[yellow],yellow,"[0.19166666666666668, 0.94, 0.5]"
47038,"(0.14444444444444443, 0.44, 0.5)","(0.9972222222222222, 0.3, 0.5)","(0.09722222222222222, 0.24, 0.5)",yellow,[yellow],yellow,"[0.14444444444444443, 0.44, 0.5]"
47039,"(0.7888888888888889, 0.24, 0.5)","(0.48333333333333334, 0.35, 0.5)","(0.8861111111111111, 0.01, 0.5)",purple,[purple],purple,"[0.7888888888888889, 0.24, 0.5]"


In [64]:
posgreentest = np.array(list(testdf[testdf.lux_last == "green"].target_hsv))

In [65]:
clf.predict_proba(posgreentest)

array([[0.67292874, 0.32707126],
       [0.74043186, 0.25956814],
       [0.72861378, 0.27138622],
       [0.67602872, 0.32397128],
       [0.6352225 , 0.3647775 ],
       [0.79518632, 0.20481368],
       [0.779819  , 0.220181  ],
       [0.72575421, 0.27424579],
       [0.73463381, 0.26536619],
       [0.77499382, 0.22500618],
       [0.66463232, 0.33536768],
       [0.67770822, 0.32229178],
       [0.71675194, 0.28324806],
       [0.51943057, 0.48056943],
       [0.66256175, 0.33743825],
       [0.79208057, 0.20791943],
       [0.6973127 , 0.3026873 ],
       [0.82644593, 0.17355407],
       [0.71002409, 0.28997591],
       [0.74048853, 0.25951147],
       [0.72717464, 0.27282536],
       [0.78653007, 0.21346993],
       [0.83340783, 0.16659217],
       [0.72151284, 0.27848716],
       [0.65192903, 0.34807097],
       [0.72993191, 0.27006809],
       [0.67394216, 0.32605784],
       [0.73955659, 0.26044341],
       [0.69191149, 0.30808851],
       [0.62587834, 0.37412166],
       [0.